# K-means and K-medoids

## Problem


Assume we have a 2D dataset consisting of (0,−6),(4,4),(0,0),(−5,2). We wish to do k-means and k-medoids clustering with k=2. We initialize the cluster centers with (−5,2),(0,−6).

For this small dataset, in choosing between two equally valid exemplars for a cluster in k-medoids, choose them with priority in the order given above (i.e. all other things being equal, you would choose (0,−6) as a center over (−5,2)).

For the following scenarios, give the clusters and cluster centers after the algorithm converges.



In [1]:
import numpy as np

In [2]:
X = np.array(([0,-6], [4,4], [0,0], [-5,2]))
centers = np.array(([-5,2], [0,-6]))
X.shape

(4, 2)

In [3]:
def dist(x_1, x_2, norm="l_1"):
    """returns distance between two points
    l_1: l_1 norm,
    l_2: l_2 norm"""
    if norm == "l_1":
        return np.linalg.norm(x_1 - x_2, ord = 1)
    elif norm == "l_2":
        return np.linalg.norm(x_1 - x_2)

In [4]:
def abs(x):
    """returns the alsolute value of scalar x"""
    if x < 0:
        return -x
    else:
        return x

In [5]:
def list_to_array(x_list):
    """return numpy array (m x n), given list of m numpy arrays (1 x n)"""
    result_arr = np.empty([len(x_list), len(x_list[0])])
    for i in range(len(x_list)):
        result_arr[i,:] = x_list[i]
    return result_arr

In [6]:
def labelled_X(X_l):
    """return labelled points (usage of temporary dummy labels)"""
    return np.append(X_l, np.zeros([len(X_l), 1]), axis = 1)

In [7]:
def medoid(X_l, norm="l_1"):
    """returns central point (not mean!),
    points X has to be labelleed in one cluster"""
    
    # append one coloumn for the distances to each point
    center_dists = np.append(X_l, np.zeros([len(X_l), 1]), axis = 1)
    
    # calculate cumulative distance for each point in cluster
    for center in center_dists:
        for point in X_l:
            center[-1] += dist(point, center[0:X_l.shape[1]], norm)
    
    # return point with minimum cumulative distance
    return center_dists[np.argmin(center_dists[:,-1])][0:X_l.shape[1]]

# quick check
test = np.ones(([1,2]))
medoid(test)

array([1., 1.])

In [8]:
def mean(X_l, norm="l_1"):
    """returns mean point,
    points X has to be labelled within one cluster
    l_1: l_1 norm and k-median clustering,
    l_2: l_2 norm and mean clustering"""
    
    # calculate the mean value of clustered points
    # return mean or median center (depending on norm)
    if norm == "l_1":
        return np.median(X_l, axis = 0)[0:X_l.shape[1]]
    elif norm == "l_2":
        return np.mean(X_l, axis = 0)[0:X_l.shape[1]]
     

# quick check
test = np.arange(0,8).reshape([2,4])
mean(test, "l_2") 

array([2., 3., 4., 5.])

In [9]:
def k_step(X_l, centers, cluster_type, norm="l_1"):
    """returns tuple of numpy array with centers and 
    list of numpy array of labelled points"""
    
    dim = X_l.shape[1] - 1
    
    #calculate distances of points to given centers
    distances = np.empty([len(X_l), len(centers)])
    i = 0
    #TODO: remove for-loops
    for point in X_l:
        j = 0
        for center in centers:
            # do not use the label in point for calculating the norm 
            distances[i, j] = dist(point[0:2], center, norm)
            j += 1
        i += 1
    
    # label points corresponding to the distances
    # label is the index of distances 
    # (no. of columns rises with no. of clusters)
    labels = np.argmin(distances, axis = 1)
    X_l[:, 2] = labels    
    
    # TODO: converting between lists and numpy arrays way to expensive
    # rearrange points in clusters here in list clusters
    clusters = []
    for label in range(len(centers)):
        points = []
        for point in X_l:          
            # use epsilon due to float value for label
            if abs(point[-1] - label) < 1E-15:
                points.append(point[0:dim])
        # index of clusters corresponds to label
        clusters.append(list_to_array(points))
        
    new_centers = centers
    
    for i in range(len(new_centers)):
        if cluster_type == "medoid":
            new_centers[i] = medoid(clusters[i], norm)
        elif cluster_type == "mean":
            new_centers[i] = mean(clusters[i], norm)
    return (X_l, new_centers)

In [10]:
# run the k-medoids algorithm
X_new = labelled_X(X)

index = 0
# hard-coded amount of while-loops, 
# TODO: convergence criteria and max stepsize 
while (index < 100):
    (X_new, centers) = k_step(X_new, centers, cluster_type="mean", norm="l_2")
    index += 1
X_new, centers

(array([[ 0., -6.,  1.],
        [ 4.,  4.,  0.],
        [ 0.,  0.,  0.],
        [-5.,  2.,  0.]]),
 array([[ 0,  2],
        [ 0, -6]]))